In [1]:
import urllib
import os
import pandas as pd
from datetime import datetime
import glob
from io import StringIO

Створити віртуальне середовище (venv) в якому будуть встановлені всі необхідні бібліотеки та налаштування для даної лабораторної роботи;

Для кожної з адміністративних одиниць України завантажити (urllib) тестові структуровані файли, що містять значення VHI-індексу. При зберіганні файлу, до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, реалізувати механізм запобігання повторного довантаження та колізії даних;

Зчитати завантажені текстові файли у pandas dataframe. Здійснити data cleaning: прибрати зайві стовпці, заповнити пропуски, видалити зайвий текст тощо. Додати стовпчики з назвою та індексом області (див. How to Handle Missing Data in Python?)

Реалізувати процедуру зміни індексів: в завантажених з NOAA даних області індексуються за англійською абеткою (Province 1 - Cherkasy), потрібно замінити індекси так, щоб області індексувались за українською абеткою (1 область - Вінницька). 


In [2]:
foldername = "provinces"
province_map = {
     1: (22, "Черкаська"),
     2: (24, "Чернігівська"),
     3: (23, "Чернівецька"),
     4: (26, "АР Крим"),
     5: (3,  "Дніпропетровська"),
     6: (4,  "Донецька"),
     7: (8,  "Івано-Франківська"),
     8: (19, "Харківська"),
     9: (20, "Херсонська"),
    10: (21, "Хмельницька"),
    11: (9,  "Київська"),
    12: (25, "м. Київ"),
    13: (10, "Кіровоградська"),
    14: (11, "Луганська"),
    15: (12, "Львівська"),
    16: (13, "Миколаївська"),
    17: (14, "Одеська"),
    18: (15, "Полтавська"),
    19: (16, "Рівненська"),
    20: (27, "м. Севастополь"),
    21: (17, "Сумська"),
    22: (18, "Тернопільська"),
    23: (6,  "Закарпатська"),
    24: (1,  "Вінницька"),
    25: (2,  "Волинська"),
    26: (7,  "Запорізька"),
    27: (5,  "Житомирська"),
}

In [3]:
def downloadprovince(id: int):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={id}&year1=1981&year2=2024&type=Mean"
    timestamp = datetime.now().strftime("%Y%m%d_%Hh%Mm%Ss")
    filename = f"Province{id}_{timestamp}.csv"
    ukr_index, ukr_name = province_map[id]
    filenameukr = f"{ukr_index}_область_{ukr_name}_{timestamp}.csv"
    #filepath = os.path.join(foldername, filename)
    filepathukr = os.path.join(foldername, filenameukr)

    try:
        print(f"[INFO] Завантаження provinceID={id}...")
        #urllib.request.urlretrieve(url, filepath)
        urllib.request.urlretrieve(url, filepathukr)
        print(f"[OK] Збережено: {filepathukr}")
    except Exception as e:
        print(f"[ERROR] Не вдалося завантажити provinceID={id}: {e}")

In [4]:
for id in range(1, 28):
    downloadprovince(id)

[INFO] Завантаження provinceID=1...
[OK] Збережено: provinces\22_область_Черкаська_20250930_17h14m11s.csv
[INFO] Завантаження provinceID=2...
[OK] Збережено: provinces\24_область_Чернігівська_20250930_17h14m25s.csv
[INFO] Завантаження provinceID=3...
[OK] Збережено: provinces\23_область_Чернівецька_20250930_17h14m27s.csv
[INFO] Завантаження provinceID=4...
[OK] Збережено: provinces\26_область_АР Крим_20250930_17h14m28s.csv
[INFO] Завантаження provinceID=5...
[OK] Збережено: provinces\3_область_Дніпропетровська_20250930_17h14m29s.csv
[INFO] Завантаження provinceID=6...
[OK] Збережено: provinces\4_область_Донецька_20250930_17h14m30s.csv
[INFO] Завантаження provinceID=7...
[OK] Збережено: provinces\8_область_Івано-Франківська_20250930_17h14m31s.csv
[INFO] Завантаження provinceID=8...
[OK] Збережено: provinces\19_область_Харківська_20250930_17h14m31s.csv
[INFO] Завантаження provinceID=9...
[OK] Збережено: provinces\20_область_Херсонська_20250930_17h14m32s.csv
[INFO] Завантаження provinceID

In [40]:
def df_by_index(ProvinceIndex):
    ukr_index, ukr_name = province_map[ProvinceIndex]
    files = glob.glob(f"provinces/{ukr_index}_область_{ukr_name}_*.csv")
    if files:
        latestfile = sorted(files)[-1]
        columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
        df= pd.read_csv(latestfile,header=None, skiprows=2, comment='<')
        del df[7]
        df = df.dropna()
        df.columns = columns
        df['province'] = ukr_name
        #display(df[~(df == -1).all(axis=1)])
        df = df[(df != -1).all(axis=1)]
    return df

df_all = pd.DataFrame()

for id in range(1, 28):
    tempdf = df_by_index(id)
    #display(tempdf)
    df_all= pd.concat([df_all, tempdf])

display(df_all)

,year,week,SMN,SMT,VCI,TCI,VHI,province
0,1982,2,0.054,262.29,46.83,31.75,39.29,Черкаська
1,1982,3,0.055,263.82,48.13,27.24,37.68,Черкаська
2,1982,4,0.053,265.33,46.09,23.91,35.00,Черкаська
3,1982,5,0.050,265.66,41.46,26.65,34.06,Черкаська
4,1982,6,0.048,266.55,36.56,29.46,33.01,Черкаська
...,...,...,...,...,...,...,...,...
2230,2024,48,0.128,270.55,64.97,25.53,45.25,Житомирська
2231,2024,49,0.115,269.06,60.12,27.24,43.68,Житомирська
2232,2024,50,0.104,267.75,55.24,25.89,40.57,Житомирська
2233,2024,51,0.094,266.45,51.16,24.29,37.72,Житомирська


Ряд VHI для області за вказаний рік

In [42]:
def vhi_by_year(df, year, province):
    return df[(df["year"] == year)&(df["province"]==province)]["VHI"]


vhi_by_year(df_all, 2014, "Львівська")


1663    46.38
1664    45.83
1665    46.55
1666    46.50
1667    46.00
1668    44.68
1669    43.35
1670    44.34
1671    46.70
1672    48.50
1673    49.37
1674    51.38
1675    51.28
1676    51.03
1677    52.35
1678    57.64
1679    62.43
1680    64.97
1681    66.26
1682    67.42
1683    67.60
1684    65.44
1685    61.38
1686    58.81
1687    55.65
1688    52.48
1689    50.21
1690    50.15
1691    50.26
1692    50.60
1693    51.12
1694    53.66
1695    56.16
1696    56.62
1697    56.36
1698    56.10
1699    54.27
1700    50.43
1701    48.70
1702    50.12
1703    48.16
1704    46.47
1705    46.32
1706    44.99
1707    47.49
1708    45.71
1709    42.65
1710    42.63
1711    42.60
1712    41.46
1713    42.42
1714    43.72
Name: VHI, dtype: float64

Ряд VHI за вказаний діапазон років для вказаних областей

In [45]:
def vhi_by_range(df, y1, y2, provincelist):
    return df[(df["year"].between(y1, y2))&(df["province"].isin(provincelist))]

plist=["Дніпропетровська", "Черкаська"]
vhi_by_range(df_all, 2000, 2014, plist)

,year,week,SMN,SMT,VCI,TCI,VHI,province
935,2000,1,0.031,257.00,20.04,51.54,35.79,Черкаська
936,2000,2,0.032,256.27,21.97,53.81,37.89,Черкаська
937,2000,3,0.031,257.02,22.44,52.47,37.46,Черкаська
938,2000,4,0.031,258.20,23.37,49.86,36.62,Черкаська
939,2000,5,0.032,258.52,23.87,51.39,37.63,Черкаська
...,...,...,...,...,...,...,...,...
1710,2014,48,0.095,270.21,51.93,28.15,40.04,Дніпропетровська
1711,2014,49,0.079,267.43,49.11,31.14,40.12,Дніпропетровська
1712,2014,50,0.067,264.59,45.60,34.89,40.24,Дніпропетровська
1713,2014,51,0.059,262.29,46.44,37.70,42.07,Дніпропетровська


Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [ ]:
def vhi_stats(df,y1, y2, provincelist):
    subset = df[(df["year"].between(y1, y2))]
    return {
        "min": subset["VHI"].min().item(),
        "max": subset["VHI"].max().item(),
        "mean": subset["VHI"].mean().item(),
        "median": subset["VHI"].median().item()
    }

plist=["Дніпропетровська", "Черкаська"]
vhi_stats(df_all,2000,2014, plist)

{'min': 5.52, 'max': 91.42, 'mean': 48.67155263157895, 'median': 48.39}

In [ ]:
#files = glob.glob("provinces/Province8_*.csv")
#latestfile = sorted(files)[-1]  
